In [73]:
import os
import re

DATA_DIR = "remote_files"

In [168]:
def get_N50(sorted_length_array):
    prefix_length = 0
    total_length = sum(sorted_length_array)
    
    for length in sorted_length_array:
        prefix_length += length
        if prefix_length >= total_length // 2:
            return length
        

def analyse(filename, description):
    lines = open(filename, "r").readlines()
    length_array = []
    
    for line in lines:
        if line.startswith(">"):
            length_array.append(int(line.split("_")[1][3:]))  # getting length from _len{num}_ in description
    
    length_array.sort(reverse=True)
    
    print(f"{description}:\n"\
          f"Amount: {len(length_array)}\n"\
          f"Sum of lengths: {sum(length_array)}\n"\
          f"Longest: {max(length_array)}\n"\
          f"N50: {get_N50(length_array)}")


def analyse_gaps(filename, description):
    lines = open(filename, "r").readlines()
    scaffolds = []
    
    ind = 0
    while ind != len(lines):
        if lines[ind].startswith(">"):
            scaffold_segments = []
            ind += 1
            while ind != len(lines) and not lines[ind].startswith(">"):
                if lines[ind].endswith("\n"):
                    scaffold_segments.append(lines[ind][:-1])
                else:
                    scaffold_segments.append(lines[ind])
                ind += 1
            
            scaffolds.append("".join(scaffold_segments))
        
    longest_scaffold_ind = max(enumerate(scaffolds), key=lambda p: len(p[1]))[0]
    longest_scaffold = scaffolds[longest_scaffold_ind]
    
    if set(longest_scaffold) == {'G', 'A', 'N', 'C', 'T'}:
        gaps = re.findall("N+", longest_scaffold)
        gap_lengths = [len(gap) for gap in gaps]

        print(f"{description}:\n"\
              f"Number of gaps: {len(gaps)}\n"\
              f"Longest gap: {max(gap_lengths)}")
    elif set(longest_scaffold) == {'G', 'A', 'C', 'T'}:
        print(f"{description}:\n"\
              "No gaps in the longest scaffold")
    else:
        print("oopsie doopsie there is an error in your program")

In [169]:
contig_file = os.path.join(DATA_DIR, "out_contig.fa")
scaffold_file = os.path.join(DATA_DIR, "out_scaffold.fa")
scaffold_gap_closed_file = os.path.join(DATA_DIR, "out_gapClosed.fa")

In [170]:
analyse(contig_file, "Contigs")

Contigs:
Amount: 508
Sum of lengths: 3922866
Longest: 304923
N50: 115749


In [171]:
analyse(scaffold_file, "Scaffolds")

Scaffolds:
Amount: 82
Sum of lengths: 3877799
Longest: 3834014
N50: 3834014


In [172]:
analyse_gaps(scaffold_file, "Original scaffolds")

Original scaffolds:
Number of gaps: 45
Longest gap: 780


In [173]:
analyse_gaps(scaffold_gap_closed_file, "Gap closed scaffolds")

Gap closed scaffolds:
Number of gaps: 8
Longest gap: 780


## Saving files to data/

In [177]:
def save_longest_scaffold(filename, out_filename):
    lines = open(filename, "r").readlines()
    out_file = open(out_filename, "w")
    scaffolds = []
    
    ind = 0
    while ind != len(lines):
        if lines[ind].startswith(">"):
            scaffold_segments = []
            ind += 1
            while ind != len(lines) and not lines[ind].startswith(">"):
                if lines[ind].endswith("\n"):
                    scaffold_segments.append(lines[ind][:-1])
                else:
                    scaffold_segments.append(lines[ind])
                ind += 1
            
            scaffolds.append("".join(scaffold_segments))
        
    longest_scaffold_ind = max(enumerate(scaffolds), key=lambda p: len(p[1]))[0]
    longest_scaffold = scaffolds[longest_scaffold_ind]
    
    out_file.write(">longest_scaffold\n")
    out_file.write(longest_scaffold)

In [126]:
!mkdir data

In [127]:
!cp remote_files/out_contig.fa data/contigs.fasta
!cp remote_files/out_gapClosed.fa data/scaffolds.fasta

In [178]:
save_longest_scaffold(scaffold_gap_closed_file, "data/longest.fasta")